# Setup

In [545]:
%pip install OpenAI

Note: you may need to restart the kernel to use updated packages.


In [546]:

from IPython.display import display, Markdown
import os
import ipywidgets as widgets

### Pretty printing help

In [547]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

### Load environment variables

In [548]:
from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

True

## Choose the Example

The following will let the user switch to a different example to run through the pipeline.

In [583]:
def set_destinations(b):
  # This function will be called when the button is clicked
  global choice, job_description_file, company_file, destination_directory, html_destination, pdf_destination, markdown_destination, word_destination
  choice = choice_widget.value
  print("You selected: " + choice)
  # source files
  job_description_file = "examples/" + choice + "/job-description.md"
  company_file = "examples/" + choice + "/about-company.md"

  # destination files
  destination_directory = "examples/" + choice + "/"
  html_destination = destination_directory + 'playbook.html'
  pdf_destination = destination_directory + 'playbook.pdf'
  markdown_destination = destination_directory + 'playbook.md'
  word_destination = destination_directory + 'playbook.docx'


# Which example to run
choice_widget = widgets.Dropdown(
  options=['medical-assistant', 'infra-engineer', 'customer-success-manager', 'real-estate-litigator','growth-manager'],
  value='medical-assistant',
  description='Choose an example:'
)
choice = choice_widget.value

# trigger the button to run the function
set_destinations(choice_widget)

button = widgets.Button(description="Confirm")


button.on_click(set_destinations)

display(choice_widget, button)



You selected: medical-assistant


Dropdown(description='Choose an example:', options=('medical-assistant', 'infra-engineer', 'customer-success-m…

Button(description='Confirm', style=ButtonStyle())

You selected: growth-manager


### Job Description

Read the job description into a variable.

In [600]:
job_description_file = "examples/" + choice + "/job-description.md"
with open(job_description_file, "r") as file:
    job_description = file.read()

print(job_description)


## Job Description

**Job Title: Growth Manager**

**Location:** Remote

**Company Overview:**
A dynamic and innovative early-stage social gaming startup, specializing in creating engaging and interactive gaming experiences. With a small, dedicated team of four, we are poised for rapid growth and are looking for a talented Growth Manager to spearhead our user acquisition and retention strategies.

**Job Description:**
We are seeking an experienced Growth Manager to join our team. This role is pivotal in driving the growth of our user base and establishing our brand in the competitive social gaming market. The ideal candidate will possess a mix of creativity, analytical prowess, and a growth hacker mindset.

**Key Responsibilities:**

1. **User Acquisition Strategy**: Develop and implement effective user acquisition strategies to attract a broad and engaged user base.
2. **Data-Driven Optimization**: Utilize data analytics to optimize marketing campaigns, understand user behavior, and 

### Company Description

Read the company description into a variable.

In [585]:

company_file = "examples/" + choice + "/about-company.md"

with open(company_file, "r") as file:
    company_description = file.read()

print(company_description)


## About SocialGameCo

**Vision Statement:**
"To revolutionize the social gaming landscape by crafting unique, immersive, and community-driven experiences that connect people globally, transcending boundaries and creating a world where play is an integral part of everyday life."

**Mission Statement:**
"Our mission is to develop innovative social gaming platforms that offer engaging, interactive, and enriching experiences. We strive to create games that not only entertain but also foster a sense of community and connection among players worldwide. By leveraging cutting-edge technology and creative storytelling, we aim to become a leading force in the social gaming industry, continuously pushing the boundaries of what social games can be."

**Values:**

1. **Innovation and Creativity:** We are committed to continuous innovation and creativity in our games and platforms. We believe in thinking outside the box to provide unique gaming experiences.

2. **Community and Connection:** Our co

# Develop top-level list of competencies

## Setup OpenAI

In [586]:
from openai import OpenAI
client = OpenAI()
llm = "gpt-4-1106-preview"

## Get Draft List of Competencies

In [587]:
prompt = f"""
You are a Talent Acquisition and Development Specialist. 
Your task is to do a job analysis and identify the competencies for a hiring team to assess in a candidate. 
Lower level jobs should have less competencies and higher level jobs should have more, but you should cover all the requirements in the job desccription. 
You should also include competencies or word competencies that are aligned with the company values and if a competency is aligned with a company value, you should say so.
Refer to the actual company name rather than 'organization' or 'company'.
Don't use the word 'technological' instead use 'technology' or 'technical'.

The job description is as follows:
{job_description}

Refer to the company description to ensure that the competencies are aligned with the company values as follows:
{company_description}

Return the key competencies for the job description as a JSON object. Use the following format:

{{
  "role": "Role Name",
  "company": "Company Name",
  "job_level": "Google Job Level (1-10)",
  "thinking": "Your thoughts about how to tackle this problem",
  "competencies": [
    {{
      "name": "Competency 1",
      "description": "Description of Competency 1",
    }},
    {{
      "name": "Competency 2",
      "description": "Description of Competency 2",
   }},
    {{
      "name": "Competency 3",
      "description": "Description of Competency 3",
    }}
  ]
}}

"""

stream = client.chat.completions.create(
    model=llm,
    max_tokens=4096,
    response_format={"type": "json_object"},
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)


output = ""

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
        output += chunk.choices[0].delta.content

competency_data = json.loads(output)
role = competency_data["role"]
company = competency_data["company"]

{
  "role": "Growth Manager",
  "company": "SocialGameCo",
  "job_level": "7",
  "thinking": "Considering the unique combination of a passion for social gaming and user growth, the role requires competencies that marry creativity with analytical skills, teamwork, and a strong understanding of marketing strategies. It's also crucial that the competencies align with SocialGameCo's values of innovation, community, excellence, integrity, learning, and fun. The role seems to be a mid- to senior-level position given the strategic responsibilities and the need for proven experience.",
  "competencies": [
    {
      "name": "Strategic User Acquisition",
      "description": "Ability to develop and execute innovative user acquisition strategies that attract and maintain a diverse and engaged user base, aligned with SocialGameCo's value of Innovation and Creativity."
    },
    {
      "name": "Analytical and Data Management",
      "description": "Strong ability to analyze performance data, pr

## Review and improve the list of competencies

In [589]:
prompt = f"""
You are a Talent Acquisition and Development Specialist. Your task is to do a job analysis and review the key competencies 
for a hiring team to look for in a candidate. Evaluate the provided list of competencies and improve them if necessary.
You will examine the provided competency framework with an eye towards comprehensiveness, relevance, and alignment with industry best practices. 
Your evaluation will focus on ensuring that the framework effectively covers all necessary areas for the {role} at {company} and 
incorporates the company's values wherever practical.
Junior or low-level jobs should have fewer competencies (e.g., 6) and more senior jobs should have more (e.g., 10).
Do not include acronyms or terminology that would not be understood by an industry professional not from the company.
Refer to the actual company name wherever possible rather than use a generic word such as 'organization' or 'company'.
Don't use the word 'technological' instead use 'technology' or 'technical'.

If you think the list of competencies is good, you can leave it as is.
If you think the list of competencies is bad, you can delete it and write a new one.
If you think the list of competencies is good but needs improvement, you can edit it.
If you think the list of competencies is good but needs more competencies, you can add more.
If you think the list of competencies is good but needs less competencies, you can delete some.
If you think the list of competencies is good but needs to be re-ordered, you can re-order them.
If you think the list of competencies is good but needs to be re-worded, you can re-word them.
If a description is aligned with a company value, you should say so.

The job description is as follows:
{job_description}

The current list of competencies is as follows:
{output}

Refer to the company description to ensure that the competencies are aligned with the company values as follows:
{company_description}

Return the improved key competency list as a JSON object. Use the following format:

{{
  "role": "Role Name",
  "company": "Company Name",
  "competencies": [
    {{
      "name": "Competency 1",
      "description": "Description of Competency 1",
      "improvements": "Improvements made (if any)"
    }},
    {{
      "name": "Competency 2",
      "description": "Description of Competency 2",
      "improvements": "Improvements made (if any)"
   }},
    {{
      "name": "Competency 3",
      "description": "Description of Competency 3",
      "improvements": "Improvements made (if any)"
    }}
  ]
}}

"""

stream = client.chat.completions.create(
    model=llm,
    max_tokens=4096,
    response_format={"type": "json_object"},
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)

output = ""

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
        output += chunk.choices[0].delta.content

competency_data = json.loads(output)


{
  "role": "Growth Manager",
  "company": "SocialGameCo",
  "competencies": [
    {
      "name": "Strategic User Acquisition",
      "description": "Ability to devise and implement user acquisition strategies that align with SocialGameCo's creative vision and commitment to community building.",
      "improvements": "Included alignment with company values of innovation, creativity, and community focus."
    },
    {
      "name": "Analytical and Data Management",
      "description": "Proficiency in leveraging data to derive actionable insights that drive marketing strategies and user growth, showcasing a commitment to quality and excellence.",
      "improvements": "Emphasized the link to SocialGameCo's value of excellence through data-driven quality."
    },
    {
      "name": "Brand Development",
      "description": "Experience in amplifying brand visibility and fostering user engagement with initiatives that reflect SocialGameCo's ethos of fun and playfulness.",
      "improvem

## Create Markdown for Competency Framework Introduction

In [606]:
markdown_competencies_intro = "\n"
counter = 1
for item in competency_data["competencies"]:
    markdown_competencies_intro += f"""
{counter}. **{item["name"]}**: {item["description"]}

"""
    counter += 1

markdown_competencies_intro = f"""

## Competency Framework

The following competencies have been identified for the {role} role at {company}:

{markdown_competencies_intro}

"""

display(Markdown(markdown_competencies_intro))



## Competency Framework

The following competencies have been identified for the Growth Manager role at SocialGameCo:



1. **Strategic User Acquisition**: Ability to devise and implement user acquisition strategies that align with SocialGameCo's creative vision and commitment to community building.


2. **Analytical and Data Management**: Proficiency in leveraging data to derive actionable insights that drive marketing strategies and user growth, showcasing a commitment to quality and excellence.


3. **Brand Development**: Experience in amplifying brand visibility and fostering user engagement with initiatives that reflect SocialGameCo's ethos of fun and playfulness.


4. **Partnership Collaboration**: Skill in nurturing strategic partnerships that expand SocialGameCo's market reach and enhance player community building.


5. **Retention Strategy Innovation**: Ability to craft and execute innovative retention strategies that keep players engaged, demonstrating commitment to SocialGameCo's values of community and quality.


6. **Budget Management**: Expertise in judiciously managing marketing resources to maximize ROI while upholding SocialGameCo's values of integrity and transparency in spendings.


7. **Cross-functional Teamwork**: Strong ability to collaborate across developmental, design, and content teams to ensure a unified user experience, reflecting SocialGameCo's value of learning and growth through teamwork.


8. **Technology Savvy**: Robust knowledge of current digital marketing technologies essential for executing growth strategies, mirroring SocialGameCo's commitment to leveraging cutting-edge innovation.


9. **Creative Problem-Solving**: Demonstrated ingenuity in tackling challenges and propelling strategic growth, aligning with SocialGameCo's core values of innovation and creativity.





## Evaluation scorecard markdown

In [591]:
markdown_evaluation_scorecard = f"""

## Interview Evaluation for {role}

| Competency | N/A | 1 | 2 | 3 | 4 | 5 |
| --- | --- | --- | --- | --- | --- | --- |
"""

counter = 1
for item in competency_data["competencies"]:
    markdown_evaluation_scorecard += f"""| {counter}. {item["name"]} | | | | | | |
"""
    counter += 1

spacer = " "

markdown_evaluation_scorecard += f"""| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | **N/A** | **1** | **2** | **3** | **4** | **5** |
| **Totals** | {spacer} | {spacer} | {spacer} | {spacer} | {spacer} | {spacer} |

"""

display(Markdown(markdown_evaluation_scorecard))



## Interview Evaluation for Growth Manager

| Competency | N/A | 1 | 2 | 3 | 4 | 5 |
| --- | --- | --- | --- | --- | --- | --- |
| 1. Strategic User Acquisition | | | | | | |
| 2. Analytical and Data Management | | | | | | |
| 3. Brand Development | | | | | | |
| 4. Partnership Collaboration | | | | | | |
| 5. Retention Strategy Innovation | | | | | | |
| 6. Budget Management | | | | | | |
| 7. Cross-functional Teamwork | | | | | | |
| 8. Technology Savvy | | | | | | |
| 9. Creative Problem-Solving | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | **N/A** | **1** | **2** | **3** | **4** | **5** |
| **Totals** |   |   |   |   |   |   |



# Detailed Competencies

## Function for analyzing a competency in depth

In [708]:
def analyze_competency(competency, competency_description):
    prompt = f"""
You are an Interviewing and Assessment Expert. Your task is to do an analysis of the competency "{competency}" for a job to give to a hiring team. 
Do not use the word 'technological' instead use 'technology' or 'technical'.

You should generate a great interview question to assess "{competency}". It should be the BEST question you could ask. 
The question should include 1-4 short follow-up questions embedded in it. 
Also produce what to look for in the answer to the lead question when evaluating the candidate's answer.

You will then generate a scenario for "{competency}" to ask the candidate to respond to. The scenario should 50 to 75 words.

You will then generate 3 subcompetencies for the "{competency}" competency. Each subcompetency should have a name and a 
question to ask the candidate. Focus on aspects of the job description or values that are not covered by the lead question.

You will then generate a set of VERY concise evaluation rubrics of less than 18 words to use when evaluating the candidate's answer 
to the lead question and the scenario.
Assume we are using a 5 point scale where 1 is the lowest rating and 5 is the highest rating.
Assume we know we are rating the candidate and don't refer to the candidate.

The competency description is currently as follows. Expand this description in your response if you think it is necessary to vividly describe the ideal candidate
but keep it to about 50 words:
{competency_description}

The job description is as follows:
{job_description}

Refer to the company description to ensure that the competencies are aligned with the company values as follows:
{company_description}

Return the competency analysis list as only a JSON object with no other commentary. Use the following format:

{{"competency": [{{
  "name": "Competency Name",
  "description": "Detailed description of the competency in 50 words",
  "lead_question": "Lead Interview Question",
  "lead_look_for": "What to look for and evaluate in the answer to the lead question. Address the interviewer.",
  "scenario": "A scenario to ask the candidate to respond to",
  "scenario_look_for": "What to look for or evaluate in the answer to the scenario. Address the interviewer.",
  "subcompetencies": [
    {{
      "name": "Subcompetency 1",
      "question": "Interview question for Subcompetency"
    }},
    {{
      "name": "Subcompetency 2",
      "question": "Interview question for Subcompetency"
    }},
    {{
      "name": "Subcompetency 3",
      "question": "Interview question for Subcompetency"
    }}
  ],
  "rating_1": "What a level 1 rating for this competency looks like. Be very concise.",
  "rating_2: "What a level 2 rating for this competency looks like. Be very concise.",
  "rating_3": "What a level 3 rating for this competency looks like. Be very concise.",
  "rating_4": "What a level 4 rating for this competency looks like. Be very concise.",
  "rating_5": "What a level 5 rating for this competency looks like. Be very concise."
}}"""

    stream = client.chat.completions.create(
        model=llm,
        max_tokens=4096,
        response_format={"type": "json_object"},
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )

    output = ""

    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")
            output += chunk.choices[0].delta.content

    return json.loads(output)


## Generate detailed competencies

This can take 3-5 minutes to generate.

In [709]:
# intialize the competency details object
competency_details = {"competencies":[]}

# set this to max number of competencies to generate. Use a lower number (e.g., 2) during testing as each competency takes 30+ seconds to generate
max_competencies = 11

# loop through the competencies and generate the competency details
for i, competency in enumerate(competency_data["competencies"]):
    if i == max_competencies:
        break
    competency_deep_dive = analyze_competency(competency["name"], competency["description"])
    competency_details["competencies"].append(competency_deep_dive)

{
  "competency": [{
    "name": "Strategic User Acquisition",
    "description": "Expertise in planning and executing initiatives to attract and retain a loyal user base aligning with SocialGameCo's vision.",
    "lead_question": "Tell me about a time when you had to develop and execute a strategic user acquisition campaign. What was your approach, how did you measure its success, and what had you learned from that experience that you would, or did, apply in subsequent strategies? Please elaborate on the strategy's alignment with the creative vision and community building.",
    "lead_look_for": "Look for specifics in planning and execution, use of data, alignment with company values, learning from outcomes, and indicators of continuous improvement.",
    "scenario": "Imagine SocialGameCo plans to launch a new game targeted at casual players aged 18-25. The goal is to acquire 100,000 users within the first month after launch. Draft a high-level strategy outline that encompasses this g

## Competency detail sections markdown

In [710]:
counter = 1
markdown_competencies = "\n"

for competency_group in competency_details["competencies"]:
  for competency_item in competency_group["competency"]:

    markdown_competencies += "## Competency " + str(counter) + ". **" 
    markdown_competencies += competency_item["name"] + "**\n\n"

    markdown_competencies += competency_item["description"] + "\n\n"

    markdown_competencies += "### Lead Question\n\n"
    markdown_competencies += "**&ldquo;" + competency_item["lead_question"] + "&rdquo;**\n\n"
    markdown_competencies += "&nbsp;\n\n"
    markdown_competencies += "&nbsp;\n\n"
    markdown_competencies += "_" + competency_item["lead_look_for"] + "_\n\n"

    markdown_competencies += "\n### Situational Scenario\n\n"
    markdown_competencies += "**&ldquo;" + competency_item["scenario"] + "&rdquo;**\n\n"
    markdown_competencies += "&nbsp;\n\n"
    markdown_competencies += "&nbsp;\n\n"
    markdown_competencies += "_" + competency_item["scenario_look_for"] + "_\n\n"

    markdown_competencies += "### Diving Deeper\n\n"
    for subcompetency in competency_item["subcompetencies"]:
      markdown_competencies += "**" + subcompetency["name"] + "**: \"" + subcompetency["question"] + "\"\n\n"
      markdown_competencies += "&nbsp;\n\n"

    markdown_competencies += "\n### Evaluation\n\n"
    markdown_competencies += "| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |\n"
    markdown_competencies += "| -------- | -------- | -------- | -------- | -------- |\n"
    markdown_competencies += "|" + competency_item["rating_1"] + "|" + competency_item["rating_2"] + "|" + competency_item["rating_3"] + "|" + competency_item["rating_4"] + "|" + competency_item["rating_5"] + "|\n\n"
    
    # markdown_competencies += "\n### Notes\n\n&nbsp;\n\n"

    counter += 1




# Final markdown

In [721]:
from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date and time in a nice readable format
formatted_now = now.strftime("%B %d, %Y %-I:%M %p")

print(formatted_now)

January 09, 2024 5:52 PM


In [722]:
# Build final markdown

final_markdown = f"""--- 
title: Interview Guide for {role}
subtitle: {company}
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: {formatted_now}
--- 

# Interview Guide for {role}

## About this Guide

Welcome to this strategic guide to streamline and enhance the interview process for the position of {role}. 
This document serves as a comprehensive resource for hiring teams, ensuring a structured, efficient, and effective evaluation of candidates. 
Through structured interviewing, our aim is to identify a candidate who not only possesses the necessary technical skills and experience but also aligns with 
{company}'s values and culture. 

The playbook outlines a clear structure for the interview process, including a detailed assessment of key competencies, practical assignments, and behavioral insights. 
Through this approach, we aspire to understand each candidate's potential and ensure the best fit for both the role and {company}.

The guide includes both behavioral and situational interview questions. We recommend dividing up competencies and questions between two or three interviewers
and using a combination of both types to assess a candidate's suitability for the role. 

- **Behavioral** questions are based on the premise that past behavior predicts future performance. These questions prompt candidates to recount specific 
past experiences, focusing on their actions and outcomes. They are effective for roles where past experience is a strong indicator of success, 
especially in skills like problem-solving and leadership. For example, asking "Describe a time when you worked under a tight deadline" assesses 
time management and pressure handling.

- **Situational** questions are hypothetical and designed to assess how candidates might handle future situations. They test judgment and problem-solving 
abilities, asking candidates to respond to imaginary scenarios. These questions are useful for evaluating less experienced candidates or roles 
requiring adaptability. An example is, "How would you handle a project falling behind schedule?" which gauges analytical thinking and adaptability.

Both behavioral and situational questions are valuable in interviews 
but serve different purposes. Behavioral questions are best for roles where past experience predicts success, while situational questions 
suit roles requiring future problem-solving and adaptability. A combination of both types offers a comprehensive assessment of a candidate's capabilities.

This interview guide was created by [Deliverables AI](https://www.deliverables.ai) using GPT-4 and is based on the job description and company description provided.
It should be used in conjunction with an interviewer's expertise and knowledge of the domain to assess the candidate's suitability for the role. 
Research shows that structured interviews are significantly more effective and less biased at identifying successful future employees and we welcome any 
feedback you have to [feedback@deliverables.ai](mailto:feedback@deliverables.ai).

{company_description}

{job_description}

{markdown_competencies_intro}

{markdown_competencies}

{markdown_evaluation_scorecard}
"""

display(Markdown(final_markdown))

# Write playbook to a text file
with open(markdown_destination, 'w') as file:
    file.write(final_markdown)

--- 
title: Interview Guide for Growth Manager
subtitle: SocialGameCo
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: January 09, 2024 5:52 PM
--- 

# Interview Guide for Growth Manager

## About this Guide

Welcome to this strategic guide to streamline and enhance the interview process for the position of Growth Manager. 
This document serves as a comprehensive resource for hiring teams, ensuring a structured, efficient, and effective evaluation of candidates. 
Through structured interviewing, our aim is to identify a candidate who not only possesses the necessary technical skills and experience but also aligns with 
SocialGameCo's values and culture. 

The playbook outlines a clear structure for the interview process, including a detailed assessment of key competencies, practical assignments, and behavioral insights. 
Through this approach, we aspire to understand each candidate's potential and ensure the best fit for both the role and SocialGameCo.

The guide includes both behavioral and situational interview questions. We recommend dividing up competencies and questions between two or three interviewers
and using a combination of both types to assess a candidate's suitability for the role. 

- **Behavioral** questions are based on the premise that past behavior predicts future performance. These questions prompt candidates to recount specific 
past experiences, focusing on their actions and outcomes. They are effective for roles where past experience is a strong indicator of success, 
especially in skills like problem-solving and leadership. For example, asking "Describe a time when you worked under a tight deadline" assesses 
time management and pressure handling.

- **Situational** questions are hypothetical and designed to assess how candidates might handle future situations. They test judgment and problem-solving 
abilities, asking candidates to respond to imaginary scenarios. These questions are useful for evaluating less experienced candidates or roles 
requiring adaptability. An example is, "How would you handle a project falling behind schedule?" which gauges analytical thinking and adaptability.

Both behavioral and situational questions are valuable in interviews 
but serve different purposes. Behavioral questions are best for roles where past experience predicts success, while situational questions 
suit roles requiring future problem-solving and adaptability. A combination of both types offers a comprehensive assessment of a candidate's capabilities.

This interview guide was created by [Deliverables AI](https://www.deliverables.ai) using GPT-4 and is based on the job description and company description provided.
It should be used in conjunction with an interviewer's expertise and knowledge of the domain to assess the candidate's suitability for the role. 
Research shows that structured interviews are significantly more effective and less biased at identifying successful future employees and we welcome any 
feedback you have to [feedback@deliverables.ai](mailto:feedback@deliverables.ai).


## About SocialGameCo

**Vision Statement:**
"To revolutionize the social gaming landscape by crafting unique, immersive, and community-driven experiences that connect people globally, transcending boundaries and creating a world where play is an integral part of everyday life."

**Mission Statement:**
"Our mission is to develop innovative social gaming platforms that offer engaging, interactive, and enriching experiences. We strive to create games that not only entertain but also foster a sense of community and connection among players worldwide. By leveraging cutting-edge technology and creative storytelling, we aim to become a leading force in the social gaming industry, continuously pushing the boundaries of what social games can be."

**Values:**

1. **Innovation and Creativity:** We are committed to continuous innovation and creativity in our games and platforms. We believe in thinking outside the box to provide unique gaming experiences.

2. **Community and Connection:** Our core focus is building strong, inclusive communities. We create games that bring people together, fostering connections and friendships.

3. **Quality and Excellence:** We are dedicated to the highest standards of quality in our game development. Our aim is to deliver excellence in every aspect of our games, from graphics to user experience.

4. **Integrity and Transparency:** We conduct our business with integrity and transparency. We believe in honest communication with our players and stakeholders.

5. **Learning and Growth:** We are committed to continuous learning and growth. We encourage our team to take on new challenges and grow both professionally and personally.

6. **Fun and Playfulness:** At our core, we believe in the power of fun and playfulness. We strive to infuse these elements into our games and our workplace culture.

By adhering to these values, we aim to not just create games, but to enhance lives through the joy of social gaming.


## Job Description

**Job Title: Growth Manager**

**Location:** Remote

**Company Overview:**
A dynamic and innovative early-stage social gaming startup, specializing in creating engaging and interactive gaming experiences. With a small, dedicated team of four, we are poised for rapid growth and are looking for a talented Growth Manager to spearhead our user acquisition and retention strategies.

**Job Description:**
We are seeking an experienced Growth Manager to join our team. This role is pivotal in driving the growth of our user base and establishing our brand in the competitive social gaming market. The ideal candidate will possess a mix of creativity, analytical prowess, and a growth hacker mindset.

**Key Responsibilities:**

1. **User Acquisition Strategy**: Develop and implement effective user acquisition strategies to attract a broad and engaged user base.
2. **Data-Driven Optimization**: Utilize data analytics to optimize marketing campaigns, understand user behavior, and drive continuous improvement.
3. **Brand Building**: Collaborate with the team to enhance brand visibility and user engagement through innovative social media and marketing campaigns.
4. **Partnership Development**: Identify and cultivate strategic partnerships to boost user growth and market presence.
5. **Retention Strategies**: Implement retention strategies to maintain a high level of user engagement and reduce churn.
6. **Budget Management**: Efficiently manage the marketing budget to maximize ROI on marketing and advertising spends.
7. **Team Collaboration**: Work closely with the development, design, and content teams to ensure a unified approach to user experience and branding.

**Qualifications:**

1. Bachelor's degree in Marketing, Business, or related field.
2. Proven experience in growth marketing, particularly in the gaming or tech industry.
3. Strong analytical skills and experience with data-driven decision making.
4. Excellent communication and teamwork skills.
5. Experience with digital marketing tools and platforms.
6. Creative thinker with a track record of implementing innovative growth strategies.

**What We Offer:**

- Opportunity to be a part of a growing startup and shape the future of social gaming.
- A collaborative, flexible, and dynamic work environment.
- Competitive compensation and equity options.
- Remote work flexibility.

**How to Apply:**
Please send your resume, a brief cover letter explaining why you are a good fit for this role, and any relevant work samples or case studies to [email@domain.com]. 

We look forward to hearing from you and potentially having you on our team to drive our growth and success!



## Competency Framework

The following competencies have been identified for the Growth Manager role at SocialGameCo:



1. **Strategic User Acquisition**: Ability to devise and implement user acquisition strategies that align with SocialGameCo's creative vision and commitment to community building.


2. **Analytical and Data Management**: Proficiency in leveraging data to derive actionable insights that drive marketing strategies and user growth, showcasing a commitment to quality and excellence.


3. **Brand Development**: Experience in amplifying brand visibility and fostering user engagement with initiatives that reflect SocialGameCo's ethos of fun and playfulness.


4. **Partnership Collaboration**: Skill in nurturing strategic partnerships that expand SocialGameCo's market reach and enhance player community building.


5. **Retention Strategy Innovation**: Ability to craft and execute innovative retention strategies that keep players engaged, demonstrating commitment to SocialGameCo's values of community and quality.


6. **Budget Management**: Expertise in judiciously managing marketing resources to maximize ROI while upholding SocialGameCo's values of integrity and transparency in spendings.


7. **Cross-functional Teamwork**: Strong ability to collaborate across developmental, design, and content teams to ensure a unified user experience, reflecting SocialGameCo's value of learning and growth through teamwork.


8. **Technology Savvy**: Robust knowledge of current digital marketing technologies essential for executing growth strategies, mirroring SocialGameCo's commitment to leveraging cutting-edge innovation.


9. **Creative Problem-Solving**: Demonstrated ingenuity in tackling challenges and propelling strategic growth, aligning with SocialGameCo's core values of innovation and creativity.






## Competency 1. **Strategic User Acquisition**

Expertise in planning and executing initiatives to attract and retain a loyal user base aligning with SocialGameCo's vision.

### Lead Question

**&ldquo;Tell me about a time when you had to develop and execute a strategic user acquisition campaign. What was your approach, how did you measure its success, and what had you learned from that experience that you would, or did, apply in subsequent strategies? Please elaborate on the strategy's alignment with the creative vision and community building.&rdquo;**

&nbsp;

&nbsp;

_Look for specifics in planning and execution, use of data, alignment with company values, learning from outcomes, and indicators of continuous improvement._


### Situational Scenario

**&ldquo;Imagine SocialGameCo plans to launch a new game targeted at casual players aged 18-25. The goal is to acquire 100,000 users within the first month after launch. Draft a high-level strategy outline that encompasses this goal.&rdquo;**

&nbsp;

&nbsp;

_Evaluate creativity, market understanding, feasibility, and alignment with company vision and community focus._

### Diving Deeper

**Data-Driven Decision Making**: "Describe the data analysis process you follow when optimizing user acquisition campaigns and provide an example of how it informed a strategy adjustment."

&nbsp;

**Innovative Brand Synergy**: "How would you integrate SocialGameCo’s values into your user acquisition strategies to ensure the brand identity resonates with our target audience?"

&nbsp;

**Partnership Leverage**: "Can you provide an example of a partnership you developed to aid user acquisition and how you ensured it was strategic and effective?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Shows a lack of relevant experience; no alignment with vision; poor understanding of metrics.|Limited experience; some strategies, but less successful; inconsistent alignment.|Moderate experience; executed strategy; learning evident; aligns somewhat with vision.|Strong experience; data-driven strategy; active learning; aligned with vision.|Exceptional experience; highly innovative, results-driven strategy; perfect brand alignment.|

## Competency 2. **Analytical and Data Management**

Proficiency in analyzing data to derive insights that inform user growth strategies, emphasizing excellence.

### Lead Question

**&ldquo;Can you describe a time when you had to analyze a complex data set to develop a marketing strategy, and how you ensured the accuracy and relevance of your analysis? What tools did you use and how did you measure success?&rdquo;**

&nbsp;

&nbsp;

_Look for a clear process for data analysis, use of appropriate tools, commitment to data accuracy, and understanding of how analysis informs strategy._


### Situational Scenario

**&ldquo;Imagine you are presented with a dataset indicating a drop in user engagement in a key demographic. How would you go about analyzing this data, and what steps would you take to address this issue?&rdquo;**

&nbsp;

&nbsp;

_Expect analytical thinking, specific steps for data analysis, and a strategic plan to remedy the engagement issue._

### Diving Deeper

**Strategic Thinking**: "How would you prioritize and align data-related activities with our company’s vision of revolutionizing the social gaming landscape?"

&nbsp;

**Marketing Acumen**: "Can you share an example of how you harnessed data to successfully pivot a marketing campaign?"

&nbsp;

**Tool Proficiency**: "Which data management or analytics platforms are you most familiar with, and how have they shaped your approach to data-driven growth?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Unable to analyze data or lacks clarity.|Basic analysis, but misses deeper insights.|Competent analysis, some strategic insight.|Strong analytical skills, clear strategy alignment.|Expert analysis, drives strategic growth decisively.|

## Competency 3. **Brand Development**

Expertise in enhancing brand visibility and engagement aligned with SocialGameCo's values of fun and playfulness.

### Lead Question

**&ldquo;Can you describe a comprehensive strategy you executed to elevate a brand's identity online? How did you measure success, and what role did cross-functional collaboration play?&rdquo;**

&nbsp;

&nbsp;

_Seek strategies that align with company values, evidence of successful metrics, and collaboration._


### Situational Scenario

**&ldquo;Imagine SocialGameCo is releasing a new game meant to bring friends together virtually. Outline a brief launch strategy that could create buzz and establish the game's brand identity.&rdquo;**

&nbsp;

&nbsp;

_Expect creative strategies that leverage company values, plans for strong initial engagement, and potential for virality._

### Diving Deeper

**Creative Campaign Execution**: "Describe how you've designed a marketing campaign that broke through the noise and was rooted in a brand's core values?"

&nbsp;

**Analytical Brand Strategy**: "Tell me about a time when you used data analytics to shape or pivot a brand's strategy. What was the outcome?"

&nbsp;

**Collaborative Brand Advocacy**: "How do you ensure brand coherence when working with external partners and influencers?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Lacks clarity on strategy, weak or no metrics, poor team integration.|Basic strategy understanding, minimal metrics, little collaboration.|Solid strategy with relevant metrics; acceptable collaboration.|Advanced strategic approach, strong metrics; good collaboration.|Exceptional strategy, impactful metrics, excels in collaboration.|

## Competency 4. **Partnership Collaboration**

Expertise in establishing and fostering strategic partnerships to boost market presence and support community engagement.

### Lead Question

**&ldquo;Can you describe a time when you successfully established a strategic partnership that had a significant impact on your organization? How did you identify the partner, negotiate the terms, and measure the success of the collaboration?&rdquo;**

&nbsp;

&nbsp;

_Assess understanding of partnership goals, negotiation skills, strategic alignment, and success metrics._


### Situational Scenario

**&ldquo;Imagine you're tasked with forming a partnership with a key influencer or brand that resonates with our target gaming audience. How would you approach them, and what would your pitch include?&rdquo;**

&nbsp;

&nbsp;

_Evaluate the approach to outreach, value proposition articulation, and alignment with SocialGameCo's vision and values._

### Diving Deeper

**Strategic Alignment**: "How do you ensure the interests of SocialGameCo and potential partners are strategically aligned before forming a collaboration?"

&nbsp;

**Negotiation Skills**: "Can you talk about a difficult partnership negotiation you faced and how you navigated it to a successful outcome?"

&nbsp;

**Relationship Building and Management**: "What strategies do you employ to maintain and deepen relationships with partners over time?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Lacks understanding of partnership building principles.|Shows basic partnership knowledge with minor success.|Demonstrates competent partnership skills, some strategic depth.|Exhibits strong partnership aptitude and success, good strategic sense.|Mastery in partnerships, exceptional strategic alignment and successes.|

## Competency 5. **Retention Strategy Innovation**

Expertise in devising and implementing inventive retention approaches to maintain an active, committed user base, aligning with our community and quality ethos.

### Lead Question

**&ldquo;Can you describe a retention strategy you've designed and implemented that was particularly innovative? How did you measure its success, and what adjustments did you make based on performance data?&rdquo;**

&nbsp;

&nbsp;

_Seek evidence of creativity, practical outcomes, adaptability, and data-informed decision-making._


### Situational Scenario

**&ldquo;Imagine social game user engagement begins to decline. How would you identify the issues and what innovative retention strategy might you deploy to reverse this trend?&rdquo;**

&nbsp;

&nbsp;

_Look for a systematic approach to problem identification, and a strategic, measurable solution._

### Diving Deeper

**Community Focused Innovation**: "How would you leverage our game’s community to increase retention and foster a sense of belonging?"

&nbsp;

**Data Utilization and Analysis**: "Detail your process for analyzing user data to inform retention strategies. Can you give an example where data led to a significant pivot in strategy?"

&nbsp;

**Collaborative Ideation**: "Describe your approach to working with cross-functional teams to generate retention strategies that align with our branding and user experience goals."

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|No concrete examples or outcomes. Lacks innovation.|Vague example, minor improvements. Little innovation.|Adequate strategy with some innovation. Limited impact shown.|Good example with clear data-informed adjustments. Shows innovation.|Excellent, proven strategy with significant impact and innovation.|

## Competency 6. **Budget Management**

Proficiency in allocating financial resources effectively to ensure optimal return on investment, adhering to SocialGameCo's ethical spending values.

### Lead Question

**&ldquo;Tell me about a time you had to manage a budget for a marketing campaign. How did you allocate the funds and what measures did you take to adapt to unforeseen costs or shifts in strategy? Were there any specific outcomes that highlighted the success or learning opportunities from your budget management?&rdquo;**

&nbsp;

&nbsp;

_Evaluate their ability to strategically allocate funds, adjust to changes, and effectively measure outcomes. Seek evidence of ethical decision-making and transparency._


### Situational Scenario

**&ldquo;Imagine you have a limited budget to acquire new users for a game launch. You need to choose between several marketing channels, some with known performance, others experimental. How would you allocate your budget?&rdquo;**

&nbsp;

&nbsp;

_Assess their strategic approach to allocation, readiness to innovate balanced with a focus on proven methods, and ability to anticipate and measure success._

### Diving Deeper

**Analytical Decision Making**: "Can you provide an example of how you've used data to inform your budgeting decisions and determine the effectiveness of spend?"

&nbsp;

**Cost Optimization**: "Describe a situation where you had to reduce the marketing budget without sacrificing significant growth opportunities. How did you approach this challenge?"

&nbsp;

**Stakeholder Management**: "How do you communicate budgetary constraints and collaborate with stakeholders to ensure alignment with the company’s growth objectives and values?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Lacks understanding of basic budgeting and ROI concepts; no ethical consideration.|Basic budget handling with minor strategic insight; minimal ethical awareness.|Solid budget management with some strategic allocation and ethical application.|Strong strategic budget oversight; balances innovation and risk; ethical transparency.|Expert budget optimizer; strong ROI focus; excels in ethics and strategic innovation.|

## Competency 7. **Cross-functional Teamwork**

Exceptional aptitude for working collaboratively with interdepartmental teams to drive unified strategies and achieve shared objectives.

### Lead Question

**&ldquo;Tell me about a time when you had to collaborate with a team or department outside your own to accomplish a project. How did you ensure your goals aligned and what steps did you take to resolve any conflicts or differences of opinion? Can you share the outcome of this collaboration?&rdquo;**

&nbsp;

&nbsp;

_Look for evidence of effective communication, the ability to bridge gaps between different team functions, negotiation skills, and evidence of a successful outcome._


### Situational Scenario

**&ldquo;Imagine our development and design teams have conflicting priorities on a project critical to user acquisition. How would you facilitate alignment and what steps would you take to ensure both teams contribute effectively to the project goals?&rdquo;**

&nbsp;

&nbsp;

_Evaluate their problem-solving skills, leadership in conflict resolution, ability to empathize with diverse perspectives, and their capacity to find a consensus._

### Diving Deeper

**Communication and Influence**: "Describe how you've managed to influence a decision or strategy that involved input from multiple departments which initially had differing viewpoints."

&nbsp;

**Problem-Solving between Teams**: "Provide an example of when you helped two teams with disparate views to reach a problem-solving consensus for the benefit of the project or organization."

&nbsp;

**Project Management with Cross-functional Teams**: "How do you prioritize tasks and manage timelines when working with teams from different functions, each with their own priorities and workloads?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Shows inability to collaborate; avoids conflict; fails to understand team dynamics.|Limited collaboration; struggles with conflict; minimal understanding of team dynamics.|Adequate collaboration; can handle some conflict; sufficient understanding of team dynamics.|Good collaborator; manages conflict well; solid grasp of team dynamics.|Excellent collaborator; resolves conflicts effectively; deep understanding of team dynamics.|

## Competency 8. **Technology Savvy**

Expertise in digital marketing technologies, essential for creatively spearheading growth strategies at SocialGameCo.

### Lead Question

**&ldquo;Can you describe an instance where you used a novel or emerging technology to solve a marketing challenge, including the outcomes and any obstacles you faced during implementation? How did you measure success?&rdquo;**

&nbsp;

&nbsp;

_Assess candidate's experience with deploying new technology solutions, ability to overcome challenges, and skills in measuring technology implementation effectiveness._


### Situational Scenario

**&ldquo;Imagine SocialGameCo is planning to enter a new market segment. How would you leverage current technology to understand this market and devise an appropriate entry strategy?&rdquo;**

&nbsp;

&nbsp;

_Look for strategic thinking, use of data analytics, and an innovative approach in leveraging technology for market analysis and strategy formulation._

### Diving Deeper

**Innovative Tool Utilization**: "How do you stay updated with the latest digital marketing tools, and can you provide an example of how you applied one such tool effectively?"

&nbsp;

**Data-Driven Decision Making**: "Explain how you have used data analytics for optimizing a marketing campaign. What metrics were most crucial to your analysis?"

&nbsp;

**Technical Adaptability**: "Tell me about a time you had to rapidly adapt to a new technology or platform change. How did you manage the transition?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Lacks experience with technology application and measurement of its impact.|Some experience but struggles to effectively leverage technology for growth.|Adequate use of technology with reasonable outcomes but lacks innovation.|Good technology implementation with measurable successes and innovative approach.|Exceptional use of cutting-edge technology to drive growth; showcases innovation, adaptability, and strategic success measurement.|

## Competency 9. **Creative Problem-Solving**

Demonstrated ingenuity in tackling challenges and propelling strategic growth, guided by innovation and creativity in alignment with the company's vision.

### Lead Question

**&ldquo;Describe a situation where you had to think outside the box to solve a growth-related problem. How did you approach the challenge, what was the outcome, and what, if anything, would you do differently now?&rdquo;**

&nbsp;

&nbsp;

_Assess creativity, practicality, results, reflection on actions, and alignment with innovation and creativity values._


### Situational Scenario

**&ldquo;Imagine our newest game’s user acquisitions are flatlining despite typical marketing methods. How would you turn this around innovatively, considering our tight budget?&rdquo;**

&nbsp;

&nbsp;

_Look for originality, practical solutions, budget-awareness, and alignment with our user growth focus._

### Diving Deeper

**Analytical Agility**: "Can you recount an instance when data revealed an unexpected insight that informed a novel marketing strategy?"

&nbsp;

**Resourcefulness**: "Describe a time when you had to implement a growth strategy with limited resources. How did you prioritize your actions?"

&nbsp;

**Strategic Adaptability**: "Tell me about a time when a well-planned strategy failed. How did you adapt and move forward?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Lacks originality, practical application; doesn’t align with company vision.|Shows minimal creativity; somewhat aligns with vision; offers basic practical solutions.|Displays moderate creativity and practical solutions; generally aligns with company values.|Demonstrates strong creative problem-solving; practical and aligns well with the company's vision.|Exemplifies exceptional creativity; innovative, budget-wise, and closely aligned with company values.|





## Interview Evaluation for Growth Manager

| Competency | N/A | 1 | 2 | 3 | 4 | 5 |
| --- | --- | --- | --- | --- | --- | --- |
| 1. Strategic User Acquisition | | | | | | |
| 2. Analytical and Data Management | | | | | | |
| 3. Brand Development | | | | | | |
| 4. Partnership Collaboration | | | | | | |
| 5. Retention Strategy Innovation | | | | | | |
| 6. Budget Management | | | | | | |
| 7. Cross-functional Teamwork | | | | | | |
| 8. Technology Savvy | | | | | | |
| 9. Creative Problem-Solving | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | **N/A** | **1** | **2** | **3** | **4** | **5** |
| **Totals** |   |   |   |   |   |   |




## Build Word docx

In [723]:
# Convert from markdown to docx
!pandoc -s {markdown_destination} -o {word_destination} --reference-doc="templates/custom-reference.docx" --toc=True --toc-depth=1 --shift-heading-level-by -1

In [724]:
# Convert from markdown to HTML

!pandoc -s {markdown_destination} -o {html_destination} --css=../../templates/html-styles.css --reference-doc="templates/html-reference.html" --toc=True --toc-depth=1 --shift-heading-level-by -1

In [725]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
HTML(html_destination).write_pdf(pdf_destination, stylesheets=['templates/html-styles.css'], full_fonts=True, hinting=True, presentational_hints=True)